In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py

from holodeck import plot
import holodeck.detstats as ds
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

In [ ]:
lib_path = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0'
hdf_name = lib_path+'/sam_lib.hdf5'
hdf_file = h5py.File(hdf_name)
print(hdf_file.keys())
# params = hdf_file['sample_params'][...]
# hc_ss = hdf_file['hc_ss'][...]
# hc_bg = hdf_file['hc_bg'][...]
# sspar = hdf_file['sspar'][...]
# bgpar = hdf_file['bgpar'][...]
fobs = hdf_file['fobs'][:]
dur = hdf_file['pta_dur'][0]
cad = hdf_file['pta_cad'][0]
hdf_file.close()

In [ ]:
npsrs = 50
sigma = 1e-7 
nskies = 25 
thresh = 0.5
nfreqs = len(fobs)

fobs_cents, fobs_edges = holo.utils.pta_freqs(dur, cad)
dfobs = np.diff(fobs_edges)[:nfreqs]
assert np.all(fobs_cents[:nfreqs] == fobs), 'fobs should match'

In [ ]:
output_dir = (lib_path+'/detstats/psrs%d_sigma%.2e'  % (npsrs, sigma))
data = ds.detect_lib(hdf_name, output_dir, npsrs=npsrs, sigma=sigma, nskies=nskies, thresh=thresh,
                     plot=True, debug=True, dur=dur, cad=cad, dfobs=dfobs, 
                     save_ssi=False, ret_dict=True)

In [ ]:
%time data1 = ds.detect_lib(hdf_name, output_dir, npsrs=npsrs, sigma=sigma, nskies=nskies, thresh=thresh, \
                     plot=True, debug=True, dur=dur, cad=cad, dfobs=dfobs, \
                     save_ssi=False, ret_dict=True)

In [ ]:
%time data2 = ds.detect_lib(hdf_name, output_dir, npsrs=npsrs, sigma=sigma, nskies=nskies, thresh=thresh, \
                     plot=True, debug=True, dur=dur, cad=cad, dfobs=dfobs, \
                     save_ssi=True, ret_dict=True)

In [ ]:
print(data['ev_ss'].shape)
print(holo.utils.stats(data['ev_ss']))

# Check script results (ss15)

In [ ]:
path = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0/detstats/psrs60_sigma2.50e-06'
filename = path+'/detstats.npz'
file = np.load(filename)
print(file.files)

In [ ]:
ev_ss = file['ev_ss']
gamma_ssi = file['gamma_ssi']
file.close()

In [ ]:
fig, ax = plot.figax(xscale='linear', xlabel='n', ylabel='expval ss')
ax.scatter(np.arange(ev_ss.size), ev_ss)

In [ ]:
print(gamma_ssi.shape)
print(holo.utils.stats(gamma_ssi[1,2,10,0]))